In [20]:
import torch
from torch import tensor
import numpy as np
from functools import partial
from opt_einsum import contract
import pyscf
from ase.calculators.singlepoint import SinglePointCalculator
from ase.units import Hartree

from pyscf import cc, gto, scf
from pyscf.scf import hf, uhf
from ase.io import read, write
from ase import Atoms
import os

In [57]:
def ae_loss(ref_dict,pred_dict, loss, **kwargs):
    """ae_loss(ref_dict, pred_dict, loss, **kwargs):
    
        Calulates atomization energy loss from reference values.

    Args:
        ref_dict ([dict]): A dictionary of reference atomization energies, whose values are flattened to a list.
        pred_dict ([dict]): A dictionary of predicted atomization energies, whose values are flattened to a list.
        loss (callable): Callable loss function
        weights (torch.Tensor) [optional]: if specified, scale individual energy differences.
            defaults to a linspace of weights from 0 to 1 of size results['E'], or 1 if only one prediction.

    Returns:
        [?]: loss called on weighted difference between reference and prediction
    """
    print("AE_LOSS FUNCTION")
    print("INPUT REF/PRED: ")
    print("REF: {}".format(ref_dict))
    print("PRED: {}".format(pred_dict))
    print("Flattening ref_dict, pred_dict")
    #ref = torch.cat(list(atomization_energies(ref_dict).values()))
    atm_pred = atomization_energies(pred_dict)
    ref = ref_dict[list(atm_pred.keys())[0]]
    pred = torch.cat(list(atm_pred.values()))
    assert len(ref) == 1
    ref = ref.expand(pred.size()[0])
    if pred.size()[0] > 1:
        weights = kwargs.get('weights', torch.linspace(0,1,pred.size()[0])**2).to(pred.device)
    else:
        weights = 1
    lae = loss((ref-pred)*weights,torch.zeros_like(pred))
    print("AE LOSS: {}".format(lae))
    return lae


def atomization_energies(energies):
    """Calculates atomization energies based on a dictionary of molecule/atomic energies.
    
    energies['ABCD'] = molecular energy
    energies['A'], energies['B'], etc. = atomic energy.
    
    Loops over ABCD - A - B - C - D

    Args:
        energies (dict): dictionary of molecule and constituent atomic energies.
    """
    def split(el):
        """Regex split molecule's symbolic expansion into constituent elements.
        No numbers must be present -- CH2 = CHH.

        Args:
            el (str): Molecule symbols

        Returns:
            list: list of individual atoms in molecule
        """
        import re
        res_list = [s for s in re.split("([A-Z][^A-Z]*)", el) if s]
        return res_list


    ae = {}
    for key in energies:
        if isinstance(energies[key],torch.Tensor):
            #if len(split(key)) == 1:continue
            e_tot = torch.clone(energies[key])
            e_tot_size = e_tot.size()
        else:
            e_tot = np.array(energies[key])
            e_tot_size = e_tot.shape
        symsplit = split(key)
        #If it is an AA reaction, energy difference between same species configurations.
        #Don't subtract A off twice, subtract AA-A for the target energy difference.
        if symsplit == ['A', 'A']:
            symsplit = ['A']
        for symbol in symsplit:
            #if single atom, continue
            if len(split(key)) == 1: continue
            e_sub = energies[symbol]
            e_sub_size = e_sub.size() if isinstance(e_sub, torch.Tensor) else e_sub.shape
            if e_tot_size == e_sub_size:
                e_tot -= e_sub
            else:
                e_tot -= e_sub[-1:]
            print('{} - {}: {}'.format(key, symbol, e_tot))
            ae[key] = e_tot
    if ae == {}:
        #empty dict -- no splitting occurred, so single atom
        ae[key] = e_tot
    print("Atomization Energy Final")
    print(ae)
    return ae
ae_loss = partial(ae_loss,loss = torch.nn.MSELoss())

In [58]:
rd = {'CCHH': tensor([-0.6459]), 'C': tensor([-37.8405]), 'H': tensor([-0.5000])}
pd = {'CCHH': tensor([-77.1652, -77.1651, -77.1651, -77.1650, -77.1650, -77.1650, -77.1650,
        -77.1650, -77.1650, -77.1650]), 'C': tensor([-37.7855, -37.7856, -37.7857, -37.7857, -37.7857, -37.7857, -37.7858,
        -37.7858, -37.7858, -37.7858]), 'H': tensor([-0.4965, -0.4966, -0.4966, -0.4967, -0.4967, -0.4967, -0.4968, -0.4968,
        -0.4968, -0.4968])}
ae_loss(rd, pd)

AE_LOSS FUNCTION
INPUT REF/PRED: 
REF: {'CCHH': tensor([-0.6459]), 'C': tensor([-37.8405]), 'H': tensor([-0.5000])}
PRED: {'CCHH': tensor([-77.1652, -77.1651, -77.1651, -77.1650, -77.1650, -77.1650, -77.1650,
        -77.1650, -77.1650, -77.1650]), 'C': tensor([-37.7855, -37.7856, -37.7857, -37.7857, -37.7857, -37.7857, -37.7858,
        -37.7858, -37.7858, -37.7858]), 'H': tensor([-0.4965, -0.4966, -0.4966, -0.4967, -0.4967, -0.4967, -0.4968, -0.4968,
        -0.4968, -0.4968])}
Flattening ref_dict, pred_dict
CCHH - C: tensor([-39.3797, -39.3795, -39.3794, -39.3793, -39.3793, -39.3793, -39.3792,
        -39.3792, -39.3792, -39.3792])
CCHH - C: tensor([-1.5942, -1.5939, -1.5937, -1.5936, -1.5936, -1.5936, -1.5934, -1.5934,
        -1.5934, -1.5934])
CCHH - H: tensor([-1.0977, -1.0973, -1.0971, -1.0969, -1.0969, -1.0969, -1.0966, -1.0966,
        -1.0966, -1.0966])
CCHH - H: tensor([-0.6012, -0.6007, -0.6005, -0.6002, -0.6002, -0.6002, -0.5998, -0.5998,
        -0.5998, -0.5998])
Atomiz

tensor(0.0005)

In [59]:
rd = {'NOO': tensor([-0.3634]), 'N': tensor([-54.5892]), 'O': tensor([-75.0673])}
pd = {'NOO': tensor([-204.8356]), 'N': tensor([-54.5133, -54.5134, -54.5136, -54.5137, -54.5137, -54.5138, -54.5138,
        -54.5138, -54.5138, -54.5138]), 'O': tensor([-74.9761, -74.9762, -74.9762, -74.9763, -74.9763, -74.9764, -74.9764,
        -74.9764, -74.9764, -74.9764])}
atomization_energies(pd)
ae_loss(rd, pd)

NOO - N: tensor([-150.3218])
NOO - O: tensor([-75.3454])
NOO - O: tensor([-0.3690])
Atomization Energy Final
{'NOO': tensor([-0.3690])}
AE_LOSS FUNCTION
INPUT REF/PRED: 
REF: {'NOO': tensor([-0.3634]), 'N': tensor([-54.5892]), 'O': tensor([-75.0673])}
PRED: {'NOO': tensor([-204.8356]), 'N': tensor([-54.5133, -54.5134, -54.5136, -54.5137, -54.5137, -54.5138, -54.5138,
        -54.5138, -54.5138, -54.5138]), 'O': tensor([-74.9761, -74.9762, -74.9762, -74.9763, -74.9763, -74.9764, -74.9764,
        -74.9764, -74.9764, -74.9764])}
Flattening ref_dict, pred_dict
NOO - N: tensor([-150.3218])
NOO - O: tensor([-75.3454])
NOO - O: tensor([-0.3690])
Atomization Energy Final
{'NOO': tensor([-0.3690])}
AE LOSS: 3.139678665320389e-05


tensor(3.1397e-05)

In [60]:
rd = {'OHH': tensor([-0.3713]), 'O': tensor([-75.0673]), 'H': tensor([-0.5000])}
pd = {'OHH': tensor([-76.3291]), 'O': tensor([-74.9773, -74.9774, -74.9775, -74.9776, -74.9776, -74.9776, -74.9776,
        -74.9776, -74.9777, -74.9777]), 'H': tensor([-0.4966, -0.4967, -0.4967, -0.4968, -0.4968, -0.4969, -0.4969, -0.4969,
        -0.4969, -0.4969])}
atomization_energies(pd)
ae_loss(rd, pd)

OHH - O: tensor([-1.3514])
OHH - H: tensor([-0.8545])
OHH - H: tensor([-0.3576])
Atomization Energy Final
{'OHH': tensor([-0.3576])}
AE_LOSS FUNCTION
INPUT REF/PRED: 
REF: {'OHH': tensor([-0.3713]), 'O': tensor([-75.0673]), 'H': tensor([-0.5000])}
PRED: {'OHH': tensor([-76.3291]), 'O': tensor([-74.9773, -74.9774, -74.9775, -74.9776, -74.9776, -74.9776, -74.9776,
        -74.9776, -74.9777, -74.9777]), 'H': tensor([-0.4966, -0.4967, -0.4967, -0.4968, -0.4968, -0.4969, -0.4969, -0.4969,
        -0.4969, -0.4969])}
Flattening ref_dict, pred_dict
OHH - O: tensor([-1.3514])
OHH - H: tensor([-0.8545])
OHH - H: tensor([-0.3576])
Atomization Energy Final
{'OHH': tensor([-0.3576])}
AE LOSS: 0.00018762654508464038


tensor(0.0002)

In [64]:
rd = {'AA': tensor([-0.1981]), 'A': tensor([0.])}
pd = {'AA': tensor([-7.4651, -7.4655, -7.4658, -7.4661, -7.4662, -7.4663, -7.4664, -7.4665,
       -7.4665, -7.4665]), 'A': tensor([-7.2674, -7.2679, -7.2683, -7.2686, -7.2688, -7.2690, -7.2691, -7.2691,
        -7.2692, -7.2692])}
ae_loss(rd, pd)

AE_LOSS FUNCTION
INPUT REF/PRED: 
REF: {'AA': tensor([-0.1981]), 'A': tensor([0.])}
PRED: {'AA': tensor([-7.4651, -7.4655, -7.4658, -7.4661, -7.4662, -7.4663, -7.4664, -7.4665,
        -7.4665, -7.4665]), 'A': tensor([-7.2674, -7.2679, -7.2683, -7.2686, -7.2688, -7.2690, -7.2691, -7.2691,
        -7.2692, -7.2692])}
Flattening ref_dict, pred_dict
AA - A: tensor([-0.1977, -0.1976, -0.1975, -0.1975, -0.1974, -0.1973, -0.1973, -0.1974,
        -0.1973, -0.1973])
Atomization Energy Final
{'AA': tensor([-0.1977, -0.1976, -0.1975, -0.1975, -0.1974, -0.1973, -0.1973, -0.1974,
        -0.1973, -0.1973])}
AE LOSS: 1.4310525386918016e-07


tensor(1.4311e-07)

In [51]:
rd = {'AB': tensor([0.0289]), 'A': tensor([0.]), 'B': tensor([0.])}
pd = {'AB': tensor([-184.9124]), 'A': tensor([-0.4967]), 'B': tensor([-184.4370])}
ae_loss(rd, pd)

AE_LOSS FUNCTION
INPUT REF/PRED: 
REF: {'AB': tensor([0.0289]), 'A': tensor([0.]), 'B': tensor([0.])}
PRED: {'AB': tensor([-184.9124]), 'A': tensor([-0.4967]), 'B': tensor([-184.4370])}
Flattening ref_dict, pred_dict
AB - A: tensor([-184.4157])
AB - B: tensor([0.0213])
Atomization Energy Final
{'AB': tensor([0.0213])}
AE LOSS: 5.774069359176792e-05


tensor(5.7741e-05)

## Testing the dimer for N2

In [24]:
#from run_pyscf.py
#spins for single atoms, since pyscf doesn't guess this correctly.
spins_dict = {
    'Al': 1,
    'B' : 1,
    'Li': 1,
    'Na': 1,
    'Si': 2 ,
    'Be':0,
    'C': 2,
    'Cl': 1,
    'F': 1,
    'H': 1,
    'N': 3,
    'O': 2,
    'P': 3,
    'S': 2
}

def get_spin(at):
    #if single atom and spin is not specified in at.info dictionary, use spins_dict
    if ( (len(at.positions) == 1) and not ('spin' in at.info) ):
        spin = spins_dict[str(at.symbols)]
    else:
        if at.info.get('spin', None):
            print('Spin specified in atom info.')
            spin = at.info['spin']
        elif 'radical' in at.info.get('name', ''):
            print('Radical specified in atom.info["name"], assuming spin 1.')
            spin = 1
        elif at.info.get('openshell', None):
            print("Openshell specified in atom info, attempting spin 2.")
            spin = 2
        else:
            print("No specifications in atom info to help, assuming no spin.")
            spin = 0
    return spin

def do_ccsdt(idx,atoms,basis, **kwargs):
    """Run a CCSD(T) (or PBE/SCAN) calculation on an Atoms object, with given basis and kwargs.

    Args:
        idx (int): Index of molecule in its trajectory. 
        atoms (ase.Atoms): The Atoms object that will be used to generate the mol object for the pyscf calculations
        basis (str): Basis type to feed to pyscf

    Raises:
        ValueError: Raised if PBC are flagged but no cell length specified.

    Returns:
        result (ase.Atoms): The Atoms object with result.calc.results['energy'] appropriately set for future use.
    """
    result = Atoms(atoms)
    pos = atoms.positions
    spec = atoms.get_chemical_symbols()
    sping = get_spin(atoms)
    #As implemented in xcdiff/dpyscf prep_data, this is not used
    #Spin is used instead.
    pol = atoms.info.get('pol', None)
    pol = None
    if kwargs.get('forcepol', False):
        pol=True
    owc = kwargs.get('owcharge', False)
    charge = atoms.info.get('charge', GCHARGE)
    print("===========================")
    print("Option Summary: {} ---> {}".format(atoms.symbols, atoms.get_chemical_formula()))
    print("Spin: {}, Polarized: {}, Charge: {}".format(sping, pol, charge))
    print("===========================")
    if owc:
        if charge != GCHARGE:
            print("OVERWRITING GCHARGE WITH ATOMS.INFO['CHARGE']. {} -> {}".format(GCHARGE, charge))

    mol_input = [[s,p] for s,p in zip(spec,pos)]
    if kwargs.get('rerun',True) and os.path.isfile('{}_{}.traj'.format(idx, atoms.symbols)):
        print("reading in traj file {}_{}.traj".format(idx, atoms.symbols))
        result = read('{}_{}.traj'.format(idx, atoms.symbols))
        return result
    print('Generating mol {} with charge {}'.format(idx, charge))
    if kwargs['PBC'] == False:
        #guess spin
        #mol = gto.M(atom=mol_input, basis=basis, charge=charge, spin=spin)
        molgen = False
        scount = 0
        while not molgen:
            try:
                mol = gto.M(atom=mol_input, basis=basis, spin=sping-scount, charge=charge)
                molgen=True
            except RuntimeError:
                #spin disparity somehow, try with one less until 0
                print("RuntimeError. Trying with reduced spin.")
                scount += 1
                if sping-scount < 0:
                    raise ValueError

        print('S: ', mol.spin)
    elif kwargs['PBC'] == True:
        if kwargs['L'] == None:
            raise ValueError('Cannot specify PBC without cell length')
        print('Generating periodic cell of length {}'.format(kwargs['L']))
        cell = np.eye(3)*kwargs['L']
        if kwargs['pseudo'] == None:
            mol = pbc.gto.M(a=cell, atom=mol_input, basis=basis, charge=charge, spin=spin)
        elif kwargs['pseudo'] == 'pbe':
            print('Assigning pseudopotential: GTH-PBE to all atoms')
            mol = pbc.gto.M(a=cell, atom=mol_input, basis=basis, charge=charge, pseudo='gth-pbe')
    if kwargs.get('testgen', None):
        print('{} Generated.'.format(atoms.get_chemical_formula()))
        return 0
    if kwargs['XC'] == 'ccsdt':
        print('CCSD(T) calculation commencing')
        #If pol specified, it's a bool and takes precedence.
        if type(pol) == bool:
            #polarization specified, UHF
            if pol:
                mf = uhf.UHF(mol)
            #specified to not polarize, RHF
            else:
                mf = hf.RHF(mol)
        #if pol is not specified in atom info, spin value used instead
        elif pol == None:
            if (mol.spin != 0):
                mf = scf.UHF(mol)
            else:
                mf = scf.RHF(mol)

        print("METHOD: ", mf)
        mf.set(chkfile='{}_{}.chkpt'.format(idx, atoms.symbols))
        if kwargs['restart']:
            print("Restart Flagged -- Setting mf.init_guess to chkfile")
            mf.init_guess = '{}_{}.chkpt'.format(idx, atoms.symbols)
        print("Running HF calculation")
        mf.run()
        print("Running CCSD calculation from HF")
        mycc = cc.CCSD(mf)
        try:
            mycc.kernel()
        except AssertionError:
            print("CCSD Failed. Stopping at HF")
            result.calc = SinglePointCalculator(result)
            ehf = (mf.e_tot) 
            etot = ehf
            eccsd = None
            eccsdt = None
            result.calc.results = {'energy': etot,
                                    'e_hf': ehf,
                                    'e_ccsd': None,
                                    'e_ccsdt':None}
            with open('progress','a') as progfile:
                progfile.write('{} \t {} \t {} \t {} \t {} \t {} \n'.format(idx, atoms.symbols, etot, ehf, eccsd, eccsdt))
            #write_mycc(idx, atoms, mf, result)
            return result

        print('MO Occ shape: ', mycc.mo_occ.shape)
        print("Running CCSD(T) calculation from CCSD")
        try:
            ccsdt = mycc.ccsd_t()
        except ZeroDivisionError:
            print("CCSD(T) Failed. DIV/0. Stopping at CCSD")
            result.calc = SinglePointCalculator(result)
            ehf = (mf.e_tot) 
            eccsd = (mycc.e_tot) 
            eccsdt = None
            etot = eccsd
            result.calc.results = {'energy': etot,
                                    'e_hf': ehf,
                                    'e_ccsd': eccsd,
                                    'e_ccsdt':eccsdt}
            with open('progress','a') as progfile:
                progfile.write('{} \t {} \t {} \t {} \t {} \t {} \n'.format(idx, atoms.symbols, etot, ehf, eccsd, eccsdt))
            #write_mycc(idx, atoms, mycc, result)
            return result

        result.calc = SinglePointCalculator(result)
        etot = (mycc.e_tot + ccsdt) 
        ehf = (mf.e_tot) 
        eccsd = (mycc.e_tot) 
        eccsdt = (ccsdt) 
        result.calc.results = {'energy': etot,
                                'e_hf': ehf,
                                'e_ccsd': eccsd,
                                'e_ccsdt': eccsdt}
        with open('progress','a') as progfile:
            progfile.write('{} \t {} \t {} \t {} \t {} \t {} \n'.format(idx, atoms.symbols, etot, ehf, eccsd, eccsdt))

        #write_mycc(idx, atoms, mycc, result)

    elif kwargs['XC'].lower() in ['pbe', 'scan']:
        print('{} calculation commencing'.format(kwargs['XC']))
            #If pol specified, it's a bool and takes precedence.
        if type(pol) == bool:
            #polarization specified, UHF
            if pol:
                mf = dft.UKS(mol)
                method = dft.UKS
            #specified to not polarize, RHF
            else:
                mf = dft.RKS(mol)
                method = dft.RKS
        #if pol is not specified in atom info, spin value used instead
        elif pol == None:
            if (mol.spin != 0):
                mf = dft.UKS(mol)
                method = dft.UKS
            else:
                mf = dft.RKS(mol)
                method = dft.RKS

        print("METHOD: ", mf)
        mf.set(chkfile='{}_{}.chkpt'.format(idx, atoms.symbols))
        if kwargs['restart']:
            print("Restart Flagged -- Setting mf.init_guess to chkfile")
            mf.init_guess = '{}_{}.chkpt'.format(idx, atoms.symbols)

        mf.grids.level = 5
        mf.max_cycle = 500
        print("Running {} calculation".format(kwargs['XC']))
        #if kwargs['df'] == False:
        #    mf = dft.RKS(mol)
        #elif kwargs['df'] == True:
        #    print('Using density fitting')
        #    mf = dft.RKS(mol).density_fit()
        mf.xc = '{},{}'.format(kwargs['XC'].lower(), kwargs['XC'].lower())
        mf.kernel()
        if not mf.converged:
            print("Calculation did not converge. Trying second order convergence with PBE to feed into calculation.")
            mfp = method(mol, xc='pbe').newton()
            mfp.kernel()
            print("PBE Calculation complete -- feeding into original kernel.")
            mf.kernel(dm0 = mfp.make_rdm1())
            if not mf.converged:
                print("Convergence still failed -- {}".format(atoms.symbols))
                with open('unconv', 'a') as f:
                    f.write('{} \t {} \t {} \n'.format(idx, atoms.symbols, mf.e_tot))

        if kwargs['df'] == True:
            print('Default auxbasis', mf.with_df.auxmol.basis)
        result.calc = SinglePointCalculator(result)
        result.calc.results = {'energy':mf.e_tot }
        with open('progress','a') as progfile:
            progfile.write('{} \t {} \t {} \n'.format(idx, atoms.symbols, mf.e_tot ))
        #np.savetxt('{}.m_occ'.format(idx), mf.mo_occ, delimiter=' ')
        #np.savetxt('{}.mo_coeff'.format(idx), mf.mo_coeff, delimiter=' ')
        #write_mycc(idx, atoms, mf, result)
    
    return result

In [26]:
p = '/home/awills/Documents/Research/swxcd/aegis/dimers/n2/'
configs = read(os.path.join(p, 'dimers.traj'), ':')
n2s = sorted([i for i in os.listdir(p) if 'N2.traj' in i],
            key=lambda x: int(x.split('_')[0]))
n2r = []
for i in n2s:
    n2r += read(os.path.join(p, i), ':')
nds = []
for idx, at in enumerate(configs):
    d = at.positions[0, -1] - at.positions[1, -1]
    nds.append(d)

In [28]:
for idx, d in enumerate(nds):
    print(idx, d)

0 0.2
1 0.23243243243243245
2 0.2648648648648649
3 0.2972972972972973
4 0.32972972972972975
5 0.3621621621621622
6 0.3945945945945946
7 0.42702702702702705
8 0.4594594594594595
9 0.4918918918918919
10 0.5243243243243243
11 0.5567567567567568
12 0.5891891891891892
13 0.6216216216216217
14 0.654054054054054
15 0.6864864864864866
16 0.7189189189189189
17 0.7513513513513514
18 0.7837837837837838
19 0.8162162162162163
20 0.8486486486486486
21 0.8810810810810812
22 0.9135135135135135
23 0.945945945945946
24 0.9783783783783784
25 1.010810810810811
26 1.0432432432432432
27 1.0756756756756758
28 1.098792
29 1.1081081081081081
30 1.1405405405405407
31 1.172972972972973
32 1.2054054054054053
33 1.2378378378378379
34 1.2702702702702704
35 1.3027027027027027
36 1.335135135135135
37 1.3675675675675676
38 1.4000000000000001
39 1.4324324324324325
40 1.4648648648648648
41 1.4972972972972973
42 1.5297297297297299
43 1.5621621621621622
44 1.5945945945945945
45 1.627027027027027
46 1.6594594594594596
47 1

In [5]:
configs

[Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(symbols='N2', pbc=False),
 Atoms(sym

In [33]:
idx = 106
GCHARGE = 0
res = do_ccsdt(idx, configs[idx], basis='6-311++G(3df,2pd)', XC='ccsdt', forcepol=True, charge=0,
              PBC=False, restart=False)

No specifications in atom info to help, assuming no spin.
Option Summary: N2 ---> N2
Spin: 0, Polarized: True, Charge: 0
Generating mol 106 with charge 0
S:  0
CCSD(T) calculation commencing
METHOD:  <pyscf.scf.uhf.UHF object at 0x7fcf7fd53a10>
Running HF calculation
converged SCF energy = -107.947382648299  <S^2> = 5.8039795e-11  2S+1 = 1
Running CCSD calculation from HF

WARN:  diis singular, eigh(h) [-2.19636569e+00 -1.57963920e-14  1.63703038e-03  5.11684450e-02
  6.50354861e-01  2.61174956e+00  1.03325413e+02]



LinAlgError: Singular matrix

In [31]:
print(res.calc.results)

{'energy': -108.74660124765416, 'e_hf': -107.9694882439877, 'e_ccsd': -108.65845558041543, 'e_ccsdt': -0.08814566723872007}


In [32]:
idx = 32
GCHARGE = 0
res = do_ccsdt(idx, configs[idx], basis='6-311++G(3df,2pd)', XC='ccsdt', forcepol=True, charge=0,
              PBC=False, restart=False)
print(res.calc.results)

No specifications in atom info to help, assuming no spin.
Option Summary: N2 ---> N2
Spin: 0, Polarized: True, Charge: 0
Generating mol 32 with charge 0
S:  0
CCSD(T) calculation commencing
METHOD:  <pyscf.scf.uhf.UHF object at 0x7fcf7fd49f10>
Running HF calculation
converged SCF energy = -108.934421697935  <S^2> = 1.4181545e-11  2S+1 = 1
Running CCSD calculation from HF
E(UCCSD) = -109.373313647828  E_corr = -0.4388919498930651
MO Occ shape:  (2, 78)
Running CCSD(T) calculation from CCSD
UCCSD(T) correction = -0.0238056165909859
{'energy': -109.39711926441896, 'e_hf': -108.93442169793491, 'e_ccsd': -109.37331364782797, 'e_ccsdt': -0.023805616590985897}
